In [1]:
import pandas as pd
import numpy as np
import vartools as vt

# En forex, elegir con base en el denominador. Si quiero pesos entonces CHFMXN es el bueno
"""
last_price = df.iloc[-1]
port_value = (last_price * n_stocks).sum()
weights = last_price * n_stocks / port_value
weights
"""

'\nlast_price = df.iloc[-1]\nport_value = (last_price * n_stocks).sum()\nweights = last_price * n_stocks / port_value\nweights\n'

In [2]:
df = pd.read_excel('data_examen.xlsx')
df = df.set_index('Date')
df

,ABT,KXI,QQQ,X,XOM
Date,,,,,
2020-01-02,79.6899,49.6289,209.9761,10.5066,55.6312
2020-01-03,78.7185,49.5841,208.0528,10.4289,55.1839
2020-01-06,79.1309,49.7366,209.3932,10.4775,55.6076
2020-01-07,78.6910,49.4136,209.3641,10.8076,55.1525
2020-01-08,79.0117,49.4046,210.9378,10.7785,54.3208
...,...,...,...,...,...
2024-12-04,115.9700,63.6400,523.2600,38.3900,114.2800
2024-12-05,115.5900,64.0400,521.8100,38.8500,114.7800
2024-12-06,115.6000,63.7700,526.4800,38.4500,113.5700


In [3]:
stocks = ['ABT','KXI','QQQ','X','XOM']
n_stocks = [5432, 8972, 3243, 7456, 5931]
long = True
conf = 95

In [4]:
vt.var_stocks(df, n_stocks, conf, long, stocks)

,Métrica,Porcentaje,cash
0,VaR,0.0177,"67,612.3411"
1,cVaR,0.0315,"120,212.2771"


In [5]:
rt = df.pct_change().dropna()
rt.cov()

,ABT,KXI,QQQ,X,XOM
ABT,0.0003,0.0001,0.0001,0.0001,0.0001
KXI,0.0001,0.0001,0.0001,0.0001,0.0001
QQQ,0.0001,0.0001,0.0003,0.0002,0.0001
X,0.0001,0.0001,0.0002,0.0015,0.0004
XOM,0.0001,0.0001,0.0001,0.0004,0.0005


In [6]:
df = df[stocks]
last_price = df.iloc[-1]
pv = np.sum(last_price * n_stocks)

weights = (last_price * n_stocks) / pv
objetivo = [0.1625, 0.4406, 0.2104, 0.0075, 0.1790]
weights

ABT   0.1643
KXI   0.1497
QQQ   0.4421
X     0.0688
XOM   0.1750
Name: 2024-12-10 00:00:00, dtype: float64

In [7]:
vt.rebalance_stocks(weights, objetivo, df, stocks, pv)

,ABT,KXI,QQQ,X,XOM
Peso Original,0.1643,0.1497,0.4421,0.0688,0.1750
Peso Óptimo,0.1625,0.4406,0.2104,0.0075,0.1790
Acciones (C/V),-60.0000,"17,435.0000","-1,700.0000","-6,644.0000",135.0000


In [8]:
df = pd.read_excel('data_examen.xlsx', sheet_name=3)
df = df.set_index('Date')
df = df[['MXNUSD','JPYUSD']]
df.head()

,MXNUSD,JPYUSD
Date,,
2020-01-01,0.0529,0.0092
2020-01-02,0.0529,0.0092
2020-01-03,0.0531,0.0092
2020-01-06,0.0528,0.0093
2020-01-07,0.0531,0.0092


In [9]:
long = False
currencies = ['MXNUSD','JPYUSD']
conf = 99
positions = [49000000, 73000000]

In [10]:
vt.var_forex(df, positions, conf, long, currencies)

,Métrica,Porcentual,Cash
0,VaR,0.0158,"46,019.0110"
1,cVaR,0.0192,"55,955.3110"


In [11]:
# Write in order S, K, r, sigma, T, N (money invested in each option)

info_call = [[2694, 2650, 0.0439, 0.31, 3/12, 21],
             [2694, 2670, 0.0439, 0.28, 3/12, 25],
             [2694, 2650, 0.0421, 0.43, 12/12, 28],
             [2694, 2670, 0.0421, 0.37, 12/12, 33]]

info_put = [[2694, 2730, 0.0439, 0.35, 3/12, 45],
             [2694, 2740, 0.0439, 0.36, 3/12, 10],
             [2694, 2730, 0.0421, 0.51, 12/12, 12],
             [2694, 2740, 0.0421, 0.58, 12/12, 43]]

In [12]:
vt.BlackScholes().delta_hedge(info_call, info_put)

19.37771011216948